In [ ]:
#Config APIs

import os
import json
import requests
from dotenv import load_dotenv

script_dir = os.getcwd()
config_path = (script_dir + '/pipeline-climate/ignore/config.json')

with open(config_path) as config_file:
    config = json.load(config_file)
    APIs = config.get('APIs').rstrip('/')

env_path = os.path.join(APIs, ".env")
load_dotenv(dotenv_path=env_path)

api_keys ={
    "openweather": os.getenv("OPENWEATHER_API_KEY", ""),
    "weatherapi": os.getenv("WEATHERAPI_API_KEY", ""),
    "weatherbit": os.getenv("WEATHERBIT_API_KEY", ""),
    "tomorrow": os.getenv("TOMORROW_API_KEY", ""),
    "iqair": os.getenv("IQAIR_API_KEY", ""),
    "openaq": os.getenv("OPENAQ_API_KEY", ""),
    "bigdata": os.getenv("BIGDATA_API_KEY", ""),
}

api_configs = {
    "openweather": {
        "url": "http://api.openweathermap.org/data/2.5/weather",
        "params": {
            "q": "", 
            "appid": api_keys["openweather"],
            "units": "metric"
        }
    },
    "weatherapi": {
        "url": "http://api.weatherapi.com/v1/current.json",
        "params": {
            "key": api_keys["weatherapi"],
            "q": "" 
        }
    },
    "weatherbit": {
        "url": "https://api.weatherbit.io/v2.0/current",
        "params": {
            "key": api_keys["weatherbit"],
            "city": ""  
        }
    },
    "tomorrow": {
        "url": "https://api.tomorrow.io/v4/weather/daily",
        "params": {
            "key": api_keys["tomorrow"],
            "location": ""
        }
    },
    "iqair": {
        "url": "https://api.airvisual.com/v2/city",
        "params": {
            "key": api_keys["iqair"],
            "city": "", 
            "state": "",  
            "country": ""  
        }
    },
    "openaq": {
        "url": "https://api.openaq.org/v2/measurements",
        "params": {
            "key": api_keys["openaq"],
            "city": "", 
        }
    },
    "bigdata": {
        "url": "https://api.bigdatacloud.net/data/reverse-geocode-client",
        "params": {
            "key": api_keys["bigdata"],
            "latitude": "",
            "longitude": ""
        }
    }
}

In [ ]:
#Reading data 
def read_api_data(api_name, *args):
    """Request of data dependending of wanted API"""
    if api_name in api_configs:
        config = api_configs[api_name]
        
        #if len(args) < len(config["params"]):
            #raise ValueError (f'Missing params for {api_name}')
        
        params = config["params"].copy()
        params_keys = list(params.keys())
        for i, key in enumerate(params_keys):
            if i < len(args):
                params[key] = args[i]

        response = requests.get(config["url"], params=params)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error to request data from API: {api_name}: {response.status_code}")
    else:
        print(f"API '{api_name}' not found")
        return None


In [ ]:
#Kafka config
from confluent_kafka import Producer

prod_conf = {
    'bootstrap.servers': 'localhost:9092, localhost:9094',
    'client.id': 'py-producer'
}

prod = Producer(**prod_conf) 

def msg_report(err, msg):
    if err is not None:
        print(f'Msg error: {msg} - {err}')
    else:
        print(f'Msg delivered to topic: {msg.topic()}, partition: [{msg.partition()}]')

def main():
    api_name = input("Write the name API wanted: ").strip().lower()
    
    if api_name in ["openweather", "weatherapi", "weatherbit", "tomorrow"]:
        location = input("Write the location: ").strip()
        weather_data = read_api_data(api_name, location)
        topic = 'Weather data'
        data = weather_data
    elif api_name == "openaq":
        location = input("Write the location: ").strip()
        air1 = read_api_data(api_name, location)
        topic = 'Air data 1'
        data = air1
    elif api_name == "iqair":
        city = input("Write the location: ").strip()
        state = input("Write the state: ").strip()
        country = input("Write the country: ").strip()
        air2 = read_api_data(api_name, city, state, country)
        data = air2
    elif api_name == "bigdata":
        latitude = float(input("Write the latitude: "))
        longitude = float(input("Write the longitude: "))
        geocode = read_api_data(api_name, latitude, longitude)
        topic = 'Geocoding data'
        data = geocode
    else:
        print("API not recognized.")
        return
    
    if data:
        print(f'Getting data from {api_name}')
        js_data = json.dumps(data).encode('utf-8')
        try:
            topic = api_name
            prod.produce(topic, key=str(data.get('id', 'unknown')), value=js_data, callback=msg_report)
            prod.poll(0) 
        except Exception as e:
            print('Error producing message: ', str((e)))
        
    prod.flush()
    
if __name__ == '__main__':
    main()
    
